# Swin Transfomer

AUCM loss

lr = 0.0015

weight decay = 0.05

too low of learning rate and too high of weight decay

In [ ]:
# Mount into drive
from google.colab import drive
drive.mount("/content/drive")
!cp '/content/drive/MyDrive/Georgia Tech/CS 7643 Deep Learning/chestx_interpretation/CheXpert-v1.0-small.zip' /content/
!mkdir CheXpert
!unzip '/content/drive/MyDrive/Georgia Tech/CS 7643 Deep Learning/chestx_interpretation/CheXpert-v1.0-small.zip' -d /content/CheXpert
import sys
sys.path.append('/content/drive/MyDrive/Georgia Tech/CS 7643 Deep Learning/chestx_interpretation/tyler')
sys.path.append('/content/drive/MyDrive/Georgia Tech/CS 7643 Deep Learning/chestx_interpretation')
sys.path.append('/content/drive/MyDrive/Georgia Tech/CS 7643 Deep Learning/chestx_interpretation/models')

Streaming output truncated to the last 5000 lines.
  inflating: /content/CheXpert/CheXpert-v1.0-small/train/patient21643/study3/view2_lateral.jpg  
  inflating: /content/CheXpert/CheXpert-v1.0-small/train/patient21643/study3/view1_frontal.jpg  
   creating: /content/CheXpert/CheXpert-v1.0-small/train/patient21643/study1/
  inflating: /content/CheXpert/CheXpert-v1.0-small/train/patient21643/study1/view1_frontal.jpg  
   creating: /content/CheXpert/CheXpert-v1.0-small/train/patient21643/study2/
  inflating: /content/CheXpert/CheXpert-v1.0-small/train/patient21643/study2/view1_frontal.jpg  
   creating: /content/CheXpert/CheXpert-v1.0-small/train/patient63218/
   creating: /content/CheXpert/CheXpert-v1.0-small/train/patient63218/study1/
  inflating: /content/CheXpert/CheXpert-v1.0-small/train/patient63218/study1/view1_frontal.jpg  
   creating: /content/CheXpert/CheXpert-v1.0-small/train/patient08046/
   creating: /content/CheXpert/CheXpert-v1.0-small/train/patient08046/study1/
  inflatin

In [ ]:
# imports
!pip install transformers
import numpy as np
import pandas as pd

%load_ext autoreload
%autoreload 2

import torch
print(torch.cuda.is_available())

from transformers import AutoFeatureExtractor, SwinForImageClassification, SwinConfig
from transformers import TrainingArguments, Trainer
from torchvision.transforms import RandomHorizontalFlip, RandomResizedCrop
from torchvision.transforms.functional import InterpolationMode

from sklearn.model_selection import train_test_split

     |████████████████████████████████| 4.0 MB 36.5 MB/s 
     |████████████████████████████████| 596 kB 66.6 MB/s 
     |████████████████████████████████| 895 kB 85.5 MB/s 
     |████████████████████████████████| 6.6 MB 64.2 MB/s 
     |████████████████████████████████| 77 kB 250 kB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
True


In [ ]:
!pip install libauc
from dataloader import *
from utils import *
from trainer import *

     |████████████████████████████████| 462 kB 52.8 MB/s 


In [ ]:
# CONSTANTS
FEATURE_EXTRACTOR_NAME = 'microsoft/swin-base-patch4-window7-224'
VIT_MODEL_NAME = 'microsoft/swin-base-patch4-window7-224'
TRAIN_SPLIT = 0.8
BATCH_SIZE = 48
LEARNING_RATE = 1.5e-3
LR_SCHEDULER_TYPE = "cosine"
WEIGHT_DECAY = 0.05
WARMUP_RATIO = 0.05
LOGGING_STRATEGY = "steps"
LOGGING_STEPS = 10
FP16 = True
EPOCHS = 10
EVALUATION_STRATEGY = "epoch"
EVAL_STEPS = 200
OUTPUT_DIR = '/content/drive/MyDrive/Georgia Tech/CS 7643 Deep Learning/chestx_interpretation/swin-chexpert-aucm-fine-tuned-output'
REMOVE_UNUSED_COLUMNS = False
GRAD_ACCUM_STEPS = 5
DATALOADER_NUM_WORKERS = 4

In [ ]:
feature_extractor = AutoFeatureExtractor.from_pretrained(FEATURE_EXTRACTOR_NAME, image_mean=[0.485, 0.456, 0.406], image_std=[0.229, 0.224, 0.225])

Downloading:   0%|          | 0.00/255 [00:00<?, ?B/s]

In [ ]:
# set up our transforms
transforms = [
    RandomResizedCrop(feature_extractor.size, scale=(0.2, 1.0), interpolation=InterpolationMode.BICUBIC),
    RandomHorizontalFlip(),
]

In [ ]:
np.random.seed(42)
train_df = pd.read_csv("/content/CheXpert/CheXpert-v1.0-small/train.csv")
train_df, eval_df = train_test_split(train_df, train_size=TRAIN_SPLIT)

train_dataset = ChexpertViTDataset("/content/CheXpert", train_df, feature_extractor, transforms=transforms, classes=COMPETITION_TASKS,
    uncertainty_method="smooth", smoothing_lower_bound=0.55, smoothing_upper_bound=0.85, use_frontal = False)
eval_dataset = ChexpertViTDataset("/content/CheXpert", eval_df, feature_extractor, classes=COMPETITION_TASKS,
    uncertainty_method="smooth", smoothing_lower_bound=0.55, smoothing_upper_bound=0.85, use_frontal = False)

In [ ]:
model = SwinForImageClassification.from_pretrained(
    VIT_MODEL_NAME,
    num_labels=len(train_dataset.labels),
    ignore_mismatched_sizes = True
).to("cuda")

Downloading:   0%|          | 0.00/70.1k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/335M [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/torch/functional.py:568: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2228.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Some weights of SwinForImageClassification were not initialized from the model checkpoint at microsoft/swin-base-patch4-window7-224 and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([1000, 1024]) in the checkpoint and torch.Size([5, 1024]) in the model instantiated
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([5]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# set up training arguments
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    evaluation_strategy=EVALUATION_STRATEGY,
    num_train_epochs=EPOCHS,
    fp16=FP16,
    eval_steps = EVAL_STEPS,
    learning_rate=LEARNING_RATE,
    remove_unused_columns=REMOVE_UNUSED_COLUMNS,
    report_to="tensorboard",
    gradient_accumulation_steps=GRAD_ACCUM_STEPS,
    lr_scheduler_type=LR_SCHEDULER_TYPE,
    weight_decay=WEIGHT_DECAY,
    warmup_ratio=WARMUP_RATIO,
    logging_strategy=LOGGING_STRATEGY,
    logging_steps=LOGGING_STEPS,
    dataloader_num_workers=DATALOADER_NUM_WORKERS
)

In [ ]:
trainer = AUCTrainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    data_collator=collate_fn,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

Using amp half precision backend


In [ ]:
trainer.train()

***** Running training *****
  Num examples = 178731
  Num Epochs = 10
  Instantaneous batch size per device = 48
  Total train batch size (w. parallel, distributed & accumulation) = 240
  Gradient Accumulation steps = 5
  Total optimization steps = 7440


Epoch,Training Loss,Validation Loss,Accuracy,F1,Auc Atelectasis,Auc Cardiomegaly,Auc Consolidation,Auc Edema,Auc Pleural effusion,Average Auc
0,0.149500,0.148742,0.123403,0.435780,0.536452,0.556725,0.544572,0.682218,0.654754,0.594944
1,0.009900,-0.007720,0.098001,0.471557,0.587515,0.585708,0.540853,0.724484,0.686628,0.625037
2,-0.005600,-0.014685,0.090482,0.488458,0.596427,0.595357,0.540804,0.733726,0.685493,0.630362
3,0.012800,-0.014196,0.077188,0.496820,0.601537,0.598681,0.548311,0.737696,0.680545,0.633354
4,-0.005900,-0.018871,0.067184,0.498325,0.600246,0.601861,0.544422,0.739089,0.676059,0.632336
5,-0.009600,-0.025439,0.056666,0.497788,0.598207,0.600945,0.553498,0.738207,0.670076,0.632186
6,-0.038800,-0.030220,0.055480,0.498356,0.597311,0.600202,0.552054,0.736499,0.667760,0.630765


Saving model checkpoint to /content/drive/MyDrive/Georgia Tech/CS 7643 Deep Learning/chestx_interpretation/swin-chexpert-aucm-fine-tuned-output/checkpoint-500
Configuration saved in /content/drive/MyDrive/Georgia Tech/CS 7643 Deep Learning/chestx_interpretation/swin-chexpert-aucm-fine-tuned-output/checkpoint-500/config.json
Model weights saved in /content/drive/MyDrive/Georgia Tech/CS 7643 Deep Learning/chestx_interpretation/swin-chexpert-aucm-fine-tuned-output/checkpoint-500/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 44683
  Batch size = 48
Saving model checkpoint to /content/drive/MyDrive/Georgia Tech/CS 7643 Deep Learning/chestx_interpretation/swin-chexpert-aucm-fine-tuned-output/checkpoint-1000
Configuration saved in /content/drive/MyDrive/Georgia Tech/CS 7643 Deep Learning/chestx_interpretation/swin-chexpert-aucm-fine-tuned-output/checkpoint-1000/config.json
Model weights saved in /content/drive/MyDrive/Georgia Tech/CS 7643 Deep Learning/chestx_interpretatio

Epoch,Training Loss,Validation Loss,Accuracy,F1,Auc Atelectasis,Auc Cardiomegaly,Auc Consolidation,Auc Edema,Auc Pleural effusion,Average Auc
0,0.149500,0.148742,0.123403,0.435780,0.536452,0.556725,0.544572,0.682218,0.654754,0.594944
1,0.009900,-0.007720,0.098001,0.471557,0.587515,0.585708,0.540853,0.724484,0.686628,0.625037
2,-0.005600,-0.014685,0.090482,0.488458,0.596427,0.595357,0.540804,0.733726,0.685493,0.630362
3,0.012800,-0.014196,0.077188,0.496820,0.601537,0.598681,0.548311,0.737696,0.680545,0.633354
4,-0.005900,-0.018871,0.067184,0.498325,0.600246,0.601861,0.544422,0.739089,0.676059,0.632336
5,-0.009600,-0.025439,0.056666,0.497788,0.598207,0.600945,0.553498,0.738207,0.670076,0.632186
6,-0.038800,-0.030220,0.055480,0.498356,0.597311,0.600202,0.552054,0.736499,0.667760,0.630765


In [ ]:
trainer.evaluate()

In [ ]:
trainer.save_state()

In [ ]:
trainer.save_model()

In [ ]:
validation_df = pd.read_csv("/content/CheXpert/CheXpert-v1.0-small/valid.csv")
valid_dataset = ChexpertViTDataset("/content/CheXpert", validation_df, feature_extractor, classes=COMPETITION_TASKS,
    uncertainty_method="smooth", smoothing_lower_bound=0.55, smoothing_upper_bound=0.85, use_frontal = False)

In [ ]:
trainer.evaluate(eval_dataset=valid_dataset)